In [1]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [2]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5
    
def fitness_function(solution):
    gains = []
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        gains.append(res[1]-res[2])
    sum_gain = sum(gains)
    defeated = sum([1 if g>=0 else 0 for g in gains])+1
    if sum_gain<0:
        final = sum_gain/defeated
    else:
        final = sum_gain*defeated
    return (final,)

"""
def fitness_function(solution):
    defeated = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        if res[2]<=0:
            defeated += 1
    return (defeated,)
"""

'\ndef fitness_function(solution):\n    defeated = 0\n    for enemy in range(1,9):\n        env.update_parameter("enemies",[enemy])\n        res = env.play(np.array(solution))\n        if res[2]<=0:\n            defeated += 1\n    return (defeated,)\n'

In [4]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1 = (alpha*np.array(parent1))+((1-alpha)*np.array(parent2))
    child2 = (alpha*np.array(parent2))+((1-alpha)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

def cxBlend(parent1,parent2,alpha):
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1,))) - alpha
    child1 = (gamma*np.array(parent1))+((1-gamma)*np.array(parent2))
    child2 = (gamma*np.array(parent2))+((1-gamma)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

In [5]:
def mutGaussian(ind,sigma,indpb):
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind))]
    return creator.Individual(np.array(ind) + np.array(mutation))

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    return creator.Individual(np.array(ind) + np.array(mutation))

In [6]:
def genetic_algorithm(initial_range,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env,creator,stats
    
    toolbox = base.Toolbox()
    toolbox.register("weight_bin", np.random.uniform,-initial_range,initial_range)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", fitness_function)

    pop = toolbox.population(n=pop_size)
    hof = {'best_solution':-np.inf,'max_value':-np.inf}

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        for ind, fit in zip(pop, pool.map(toolbox.evaluate,pop)):
            ind.fitness.values = fit

        # record best solution
        for ind in pop:
            if ind.fitness.values[0]>hof['max_value']:
                hof['best_solution'] = ind
                hof['max_value'] = ind.fitness.values[0]

        logbook = tools.Logbook()
        logbook.header = ['gen', 'nevals'] + (stats.fields if stats else []) + ['max_hof']
        record = stats.compile(pop) if stats else {}

        logbook.record(gen=0, nevals=len(pop), **record, max_hof=hof['max_value'])
        if verbose==1:
            print(logbook.stream)

        for g in range(ngen):
            # select parents
            parents = tools.selTournament(individuals=pop,k=n_offspring*pop_size,tournsize=tournsize_parent)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)       

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            for ind, fit in zip(invalid_ind, pool.map(toolbox.evaluate,invalid_ind)):
                    ind.fitness.values = fit

            # record best solution
            for ind in offspring:
                if ind.fitness.values[0]>hof['max_value']:
                    hof['best_solution'] = ind
                    hof['max_value'] = ind.fitness.values[0]

            # select the next generation of the population
            pop = tools.selTournament(individuals=offspring,k=pop_size,tournsize=tournsize_survival)

            record = stats.compile(pop) if stats else {}
            logbook.record(gen=g+1, nevals=len(invalid_ind), **record, max_hof=hof['max_value'])
            if verbose==1:
                print(logbook.stream)
        pool.close()
        pool.join()

    return hof,logbook

In [7]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Std", np.std)

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5704:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


In [8]:
def objective(trial):
    
    params = {'initial_range':trial.suggest_int("initial_range", 1,3,step=1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 30,30,step=20),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning
    iteration_logs = []
    for i in range(3):
        hof,logs = genetic_algorithm(**params)
        logs_pd = pd.DataFrame(logs)
        iteration_logs.append(logs_pd)
        hof_tuning.append(hof)
    
    return pd.concat(iteration_logs)['Max'].mean()


In [9]:
hof_tuning = []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=500)

[I 2023-10-15 00:19:14,958] A new study created in memory with name: GA


gen	nevals	Mean    	Max   	Std    	max_hof
0  	50    	-665.394	-137.4	196.919	-137.4 
1  	50    	-201.767	-91.4 	126.237	-91.4  
2  	50    	-82.3617	-49.85	22.1974	-49.85 
3  	50    	-97.3047	-48.85	82.7362	-48.85 
4  	50    	-60.0247	-22.08	25.9414	-22.08 
5  	48    	-56.5783	-22.08	26.9133	-22.08 
6  	50    	-53.3881	-12.44	26.5806	-12.44 
7  	50    	-62.5663	-30.3 	28.8389	-12.44 
8  	50    	-43.3907	-2.16667	28.8599	-2.16667
9  	49    	-39.1132	-2.16667	34.4971	-2.16667
10 	49    	-24.1133	-2.16667	29.5685	-2.16667
11 	50    	-45.3184	-20.88  	17.6847	-2.16667
12 	50    	-44.0667	-23     	16.1185	-2.16667
13 	50    	-46.0346	-9.68   	24.6862	-2.16667
14 	49    	-36.6206	-18.88  	20.744 	-2.16667
15 	50    	-26.9668	-18.88  	13.7142	-2.16667
16 	50    	-26.4121	-18.88  	10.3964	-2.16667
17 	50    	-30.03  	-9.92   	15.6371	-2.16667
18 	50    	-38.9523	-17.8   	23.056 	-2.16667
19 	50    	-38.1926	-22.56  	14.1384	-2.16667
20 	49    	-34.4509	-14.08  	19.8125	-2.16667
21 	49    	-32.

[I 2023-10-15 00:28:53,749] Trial 0 finished with value: -14.688960573476365 and parameters: {'initial_range': 2, 'pop_size': 50, 'n_offspring': 1, 'tournsize_parent': 3, 'cxpb': 0.9, 'cx_type': 'blend', 'alpha': 0.8, 'mutpb': 0.9, 'mut_type': 'adaptive', 'sigma': 2.5, 'indpb': 0.5, 'tau': 0.7000000000000001, 'tournsize_survival': 8, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: -14.688960573476365.


30 	48    	56.4252 	214     	113.404	214    
gen	nevals	Mean    	Max     	Std    	max_hof 
0  	50    	-653.895	-90.4667	214.683	-90.4667
1  	226   	-119.358	-27.1   	49.815 	-27.1   
2  	218   	-54.2052	-23.55  	21.5469	-23.55  
3  	216   	-26.461 	-23.55  	5.34218	-23.55  
4  	227   	-24.5467	-21.8   	7.23082	-21.8   
5  	217   	-23.669 	-21.8   	1.63658	-21.8   
6  	229   	-24.31  	-21.8   	4.07397	-21.8   
7  	227   	-21.8   	-21.8   	3.55271e-15	-21.8   
8  	221   	-21.8   	-21.8   	3.55271e-15	-21.8   
9  	213   	-22.042 	-21.6   	1.7228     	-21.6   
10 	215   	-37.7425	-15.36  	71.1075    	-15.36  
11 	226   	-13.1776	-5.84   	6.89776    	-5.84   
12 	229   	-8.85793	-5.84   	12.5665    	-5.84   
13 	212   	-5.84   	-5.84   	8.88178e-16	-1.76   
14 	235   	-5.84   	-5.84   	8.88178e-16	-1.76   
15 	230   	-5.7282 	-0.25   	0.7826     	-0.25   
16 	202   	-6.76613	-0.25   	9.76531    	-0.25   
17 	224   	-3.91167	-0.25   	12.7778    	-0.25   
18 	214   	-2.44867	-0.25   	10.9523 

[I 2023-10-15 01:11:14,833] Trial 1 finished with value: 26.738494623656532 and parameters: {'initial_range': 1, 'pop_size': 50, 'n_offspring': 5, 'tournsize_parent': 10, 'cxpb': 0.8, 'cx_type': 'whole_aritmetic', 'alpha': 0.8, 'mutpb': 0.4, 'mut_type': 'adaptive', 'sigma': 3.0, 'indpb': 0.1, 'tau': 0.2, 'tournsize_survival': 5, 'ngen': 30, 'verbose': 1}. Best is trial 1 with value: 26.738494623656532.


...

30 	202   	547     	547   	1.13687e-13	547    
gen	nevals	Mean    	Max  	Std    	max_hof
0  	50    	-632.963	-71.8	233.829	-71.8  
1  	202   	-100.282	-25.36	38.7484	-25.36 
2  	216   	-60.7784	-24.9 	30.1565	-24.9  
3  	192   	-27.2933	-24.9 	9.12479	-24.9  
4  	193   	-28.368 	-24.9 	24.276 	-24.9  
5  	196   	-27.7453	-24.9 	14.4016	-24.9  
6  	191   	-24.9   	-24.9 	3.55271e-15	-24.9  
7  	210   	-24.9   	-24.9 	3.55271e-15	-10.12 
8  	185   	-19.2978	110   	26.4261    	110    
9  	206   	52.15   	110   	69.8493    	110    
10 	195   	110     	110   	1.42109e-14	110    
11 	208   	103.603 	110   	31.3426    	110    
12 	209   	110     	110   	1.42109e-14	110    
13 	197   	111.98  	129   	5.37583    	129    
14 	203   	118.076 	214   	43.6471    	214    
15 	195   	145.898 	214   	68.3161    	214    
16 	200   	214     	214   	0          	214    
17 	203   	214     	214   	0          	214    
18 	207   	214     	214   	0          	214    
19 	194   	208.143 	214   	41.0013    	214 

[I 2023-10-15 10:12:24,659] Trial 21 finished with value: 85.1761290322591 and parameters: {'initial_range': 2, 'pop_size': 50, 'n_offspring': 5, 'tournsize_parent': 8, 'cxpb': 0.5, 'cx_type': 'blend', 'alpha': 'random', 'mutpb': 0.6, 'mut_type': 'normal', 'sigma': 1.5, 'indpb': 0.5, 'tau': 0.30000000000000004, 'tournsize_survival': 9, 'ngen': 30, 'verbose': 1}. Best is trial 14 with value: 293.86813620071945.


30 	207   	-5.792  	-3.8  	13.944     	-3.8   
gen	nevals	Mean    	Max  	Std    	max_hof
0  	50    	-616.205	-61.8	237.101	-61.8  
1  	244   	-138.07 	-36.36	133.801	-36.36 
2  	234   	-79.436 	-9.8  	75.9577	-9.8   
3  	245   	-71.2386	-9.8  	46.3998	-9.8   
4  	242   	-49.1027	-9.8  	41.4095	-9.8   
5  	245   	-30.9592	-9.8  	28.1619	-9.8   
6  	243   	-42.9921	-9.8  	34.8497	-9.8   
7  	241   	-44.1503	-9.8  	47.8984	-9.8   
8  	243   	-45.5776	-9.8  	52.5555	-9.8   
9  	238   	-41.4398	-9.8  	35.2228	-4.63333
10 	242   	-25.4889	-2.56 	26.5614	-2.56   
11 	246   	-46.1344	-9.8  	78.4658	-2.56   
12 	243   	-29.0442	40    	30.4261	40      
13 	243   	-35.3478	40    	37.3928	40      
14 	245   	-30.9337	40    	38.8924	40      
15 	245   	-17.9527	40    	40.9896	40      
16 	239   	-3.63953	40    	45.1256	40      
17 	239   	-11.755 	40    	37.4847	40      
18 	240   	0.888267	40    	44.2114	40      
19 	241   	-7.66227	40    	44.1496	40      
20 	244   	-12.705 	40    	41.6677	40    

Process ForkPoolWorker-275:
Process ForkPoolWorker-273:
Process ForkPoolWorker-276:
Process ForkPoolWorker-274:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    resu

KeyboardInterrupt: 

In [ ]:
#Trial 14 finished with value: 293.86813620071945 and parameters: {'initial_range': 2, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 9, 'cxpb': 0.6, 'cx_type': 'blend', 'alpha': 0.1, 'mutpb': 0.8, 'mut_type': 'normal', 'sigma': 1.0, 'indpb': 0.7000000000000001, 'tau': 0.9, 'tournsize_survival': 7, 'ngen': 30, 'verbose': 1}

In [10]:
res_pd = pd.DataFrame(hof_tuning).sort_values('max_value',ascending=False).reset_index()

In [12]:
res_pd.to_csv("v2_v2_res.csv")

In [ ]:
"""
params = {'initial_range':,
          'pop_size':,
          'n_offspring':,
          'tournsize_parent':,
          'cxpb':,
          'cx_type':,
          'alpha':,
          'mutpb':,
          'mut_type':,
          'sigma':,
          'indpb':,
          'tau':,
          'tournsize_survival':,
          'ngen':,
          'verbose':}
""""